## Project Overview: AI-Powered Real-Time Weapon Detection and Alert System

#### Problem Statement

In many homes and small businesses, especially in regions with limited access to advanced security infrastructure, there is an increasing need for affordable yet effective surveillance solutions. Traditional CCTV systems merely record footage without offering real-time response or threat detection capabilities. As a result, dangerous situations such as weapon threats may go unnoticed until it's too late. Moreover, professional security systems with instant alert mechanisms are often too expensive or require significant setup and monitoring.

#### Proposed Solution
This project introduces a low-cost, AI-powered real-time surveillance system designed to detect weapons through a standard webcam. It leverages computer vision, AI and machine learning to analyze video feeds continuously and identify the presence of a weapon with high accuracy.

Upon detection, the system sends an instant WhatsApp alert to the homeowner or business owner, including:

A short warning message (with weapon type, confidence level, and timestamp),

A snapshot of the detected frame uploaded securely using a hosting service.



In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# dataset folder
dataset_path = 'weapon_dataset'

# (80% train, 20% validation)
batch_size = 32
img_size = (224, 224) 

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 200 files belonging to 2 classes.
Using 160 files for training.
Found 200 files belonging to 2 classes.
Using 40 files for validation.
Classes: ['Knife', 'Machete']


In [2]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [3]:
from tensorflow.keras import layers, models

base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False  # Freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
epochs = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.5758 - loss: 0.9724 - val_accuracy: 0.5000 - val_loss: 0.8721
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6580 - loss: 0.7020 - val_accuracy: 0.7000 - val_loss: 0.4938
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6829 - loss: 0.6042 - val_accuracy: 0.8000 - val_loss: 0.4192
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7832 - loss: 0.4474 - val_accuracy: 0.8250 - val_loss: 0.3677
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 960ms/step - accuracy: 0.8964 - loss: 0.3411 - val_accuracy: 0.8750 - val_loss: 0.3825
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8428 - loss: 0.3718 - val_accuracy: 0.9000 - val_loss: 0.2833
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 986ms/step - accuracy: 0.9093 - loss: 0.2840 - val_accuracy: 0.9500 - val_loss: 0.2894
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9119 - loss: 0.2820 - val_accuracy: 0.9250 - val_loss: 0.2268
Epoch 9/1

In [5]:
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.2%}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - accuracy: 0.9458 - loss: 0.2022
Validation Accuracy: 95.00%


In [6]:
model.save("weapon_detector.keras")